In [93]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import (StructField,StructType, BooleanType, DoubleType,LongType)
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, to_timestamp, unix_timestamp,udf
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType, FloatType, DataType, TimestampType, Row
import inspect
from pyspark.sql.functions import lit

In [117]:
rdd_data = spark.read.option("header",True).option("inferSchema",True).csv("VGP-week3-data.csv")

In [118]:
data_schema = rdd_data.schema
print(data_schema)
def remove_header(itr_index, itr):
    return iter(list(itr)[1:]) if itr_index == 0 else itr

#rdd_data = rdd_data.mapPartitionsWithIndex(remove_header)
#rdd_data = rdd_data.filter(lambda line : line != data_schema)

# print(rdd_data)
# data_schema[5]="PM25"

# data_schema[7]="PM100"

rdd_data.printSchema()

StructType(List(StructField(kit_id,IntegerType,true),StructField(participant_id,IntegerType,true),StructField(time,StringType,true),StructField(lat,DoubleType,true),StructField(lon,DoubleType,true),StructField(PM2-5,StringType,true),StructField(PM10,StringType,true),StructField(PM1-0,StringType,true),StructField(NO2,StringType,true),StructField(BC,StringType,true),StructField(activity,StringType,true),StructField(event,StringType,true)))
root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- PM2-5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM1-0: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- BC: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)



In [119]:
rdd_data.first()

Row(kit_id=80, participant_id=9999964, time='2019-11-14 09:00:00', lat=48.7717766666667, lon=2.006005, PM2-5='NULL', PM10='NULL', PM1-0='NULL', NO2='NULL', BC='NULL', activity='NULL', event='NULL')

In [120]:
rdd_data = rdd_data.withColumnRenamed('PM2-5','PM2_5')
rdd_data = rdd_data.withColumnRenamed('PM1-0','PM1_0')
# convert to float 
#rdd_data = rdd_data.map(lambda l : [int(l[0]),int(l[1]),l[2],to_float(l[3]),to_float(l[4]),to_float(l[5]),to_float(l[6]),to_float(l[7]),to_float(l[8]),to_float(l[9]),l[10],l[11]])
rdd_data.first()
#On change les string "NULL" en null
rdd_data = rdd_data.withColumn(("BC"), when(col("BC") == "NULL", None).otherwise(col("BC")))
rdd_data = rdd_data.withColumn(("PM2_5"), when(col("PM2_5") == "NULL", None).otherwise(col("PM2_5")))
rdd_data = rdd_data.withColumn(("PM10"),  when(col("PM10") == "NULL",  None).otherwise(col("PM10")))
rdd_data = rdd_data.withColumn(("PM1_0"), when(col("PM1_0") == "NULL", None).otherwise(col("PM1_0")))
rdd_data = rdd_data.withColumn(("NO2"),   when(col("NO2") == "NULL",   None).otherwise(col("NO2")))
rdd_data = rdd_data.withColumn(("activity"), when(col("activity") == "NULL", None).otherwise(col("activity")))
rdd_data = rdd_data.withColumn(("event"), when(col("event") == "NULL", None).otherwise(col("event")))
#rdd_data = rdd_data.map(lambda l : [int(l[0]),int(l[1]),l[2],to_float(l[3]),to_float(l[4]),to_float(l[5]),to_float(l[6]),to_float(l[7]),to_float(l[8]),to_float(l[9]),l[10],l[11]])
rdd_data.first()

Row(kit_id=80, participant_id=9999964, time='2019-11-14 09:00:00', lat=48.7717766666667, lon=2.006005, PM2_5=None, PM10=None, PM1_0=None, NO2=None, BC=None, activity=None, event=None)

In [126]:
df_data = rdd_data
# df_data.show()
df_data.where(col('time').isNull()).show()

+------+--------------+----+---+---+-----+----+-----+---+---+--------+-----+
|kit_id|participant_id|time|lat|lon|PM2_5|PM10|PM1_0|NO2| BC|activity|event|
+------+--------------+----+---+---+-----+----+-----+---+---+--------+-----+
+------+--------------+----+---+---+-----+----+-----+---+---+--------+-----+



In [99]:
df_data.printSchema()

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- PM2_5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM1_0: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- BC: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)



In [116]:
df_data.show(5)

+------+--------------+-------------------+----------------+--------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|               time|             lat|     lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|
+------+--------------+-------------------+----------------+--------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|2019-11-14 00:00:00|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|2019-11-14 00:00:00|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|2019-11-14 00:00:00|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|2019-11-14 00:00:00|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|2019-11-14 00:00:00|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
+------+--------------+-------------------+----------------+--------+-----+----+-----+----+----+--------

In [128]:
df_data = df_data.withColumn('time', unix_timestamp('time','yyyy-MM-dd HH:mm:ss').alias('time'))

AnalysisException: cannot resolve 'unix_timestamp(`time`, 'yyyy-MM-dd HH:mm:ss')' due to data type mismatch: argument 1 requires (string or date or timestamp) type, however, '`time`' is of bigint type.;;
'Project [kit_id#3386, participant_id#3387, unix_timestamp(time#3795L, yyyy-MM-dd HH:mm:ss, Some(Europe/Paris)) AS time#3882, lat#3389, lon#3390, PM2_5#3473, PM10#3486, PM1_0#3499, NO2#3512, BC#3460, activity#3525, event#3538]
+- Project [kit_id#3386, participant_id#3387, unix_timestamp(time#3388, yyyy-MM-dd HH:mm:ss, Some(Europe/Paris)) AS time#3795L, lat#3389, lon#3390, PM2_5#3473, PM10#3486, PM1_0#3499, NO2#3512, BC#3460, activity#3525, event#3538]
   +- Project [kit_id#3386, participant_id#3387, time#3388, lat#3389, lon#3390, PM2_5#3473, PM10#3486, PM1_0#3499, NO2#3512, BC#3460, activity#3525, CASE WHEN (event#3397 = NULL) THEN cast(null as string) ELSE event#3397 END AS event#3538]
      +- Project [kit_id#3386, participant_id#3387, time#3388, lat#3389, lon#3390, PM2_5#3473, PM10#

In [159]:
df_data.where(col('time').isNotNull()).show()
df_data.printSchema()

+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|      time|             lat|             lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|
+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|1573718400|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718410|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718420|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718430|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718440|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718450|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|

In [131]:
df_data.filter(df_data["participant_id"] == 9999920).filter(df_data["PM2_5"].isNotNull()).count()

1505

In [132]:
# Test cell Amir
df_data.printSchema()
df_data.sort('time').show()

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- PM2_5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM1_0: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- BC: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)

+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|      time|             lat|             lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|
+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|1573718400|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718410|48.7717766666667|        2.006005| null|null| null|null|null|    null|

In [160]:
Ureal_format = [StructField('a', IntegerType(),True),
              StructField('b', DoubleType(),True),
              StructField('c', DoubleType(),True),
              StructField('r', BooleanType(),True),
              StructField('t1', LongType(),True),
              StructField('t2', LongType(),True)]
UrealType = StructType(Ureal_format)

In [161]:
Position_format = [StructField('lat', DoubleType(),True),
              StructField('lon', DoubleType(),True)]
PositionType = StructType(Position_format)

In [162]:
USreal_format = [StructField('a', IntegerType(),True),
              StructField('b', DoubleType(),True),
              StructField('c', DoubleType(),True),
              StructField('r', BooleanType(),True),
              StructField('p1', PositionType,True),
              StructField('p2', PositionType,True)]
USrealType = StructType(USreal_format)

In [163]:
Mreal_format = [StructField('time_units', ArrayType(UrealType),True)]
MrealType = StructType(Mreal_format)

In [164]:
MSreal_format = [StructField('space_units', ArrayType(USrealType),True)]
MSrealType = StructType(MSreal_format)

In [138]:
PM100_temp = spark.createDataFrame(sc.emptyRDD(), StructType([StructField("id_participant", IntegerType(), False), StructField("PM100_profil_temp", MrealType, True)]))

In [158]:
PM100_temp.printSchema()

root
 |-- id_participant: integer (nullable = false)
 |-- PM100_profil_temp: struct (nullable = true)
 |    |-- time_units: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: integer (nullable = true)
 |    |    |    |-- b: double (nullable = true)
 |    |    |    |-- c: double (nullable = true)
 |    |    |    |-- r: boolean (nullable = true)
 |    |    |    |-- t1: timestamp (nullable = true)
 |    |    |    |-- t2: timestamp (nullable = true)



In [140]:
PM100_test = df_data.filter(df_data["PM1_0"].isNotNull()).filter(df_data["time"].isNotNull()).select("participant_id","time","PM1_0")

In [141]:
PM100_test.show(5)

+--------------+----------+-----+
|participant_id|      time|PM1_0|
+--------------+----------+-----+
|       9999964|1573804980|    4|
|       9999964|1573804990|    4|
|       9999964|1573805000|    4|
|       9999964|1573805010|    4|
|       9999964|1573805020|    4|
+--------------+----------+-----+
only showing top 5 rows



In [142]:
PM100_test = PM100_test.groupBy("participant_id").agg(F.collect_list(col('time')).alias('time'),F.collect_list(col('PM1_0')).alias('PM1_0'))

In [144]:
PM100_test.head()

Row(participant_id=9999920, time=[1573810600, 1573810610, 1573810620, 1573810630, 1573810640, 1573810650, 1573810660, 1573810670, 1573810680, 1573810690, 1573810700, 1573810710, 1573810720, 1573810730, 1573810740, 1573810750, 1573810760, 1573810770, 1573810780, 1573810790, 1573810800, 1573810810, 1573810820, 1573810830, 1573810840, 1573810850, 1573810860, 1573810870, 1573810880, 1573810890, 1573810900, 1573810910, 1573810920, 1573810930, 1573810940, 1573810950, 1573810960, 1573810970, 1573810980, 1573810990, 1573811000, 1573811010, 1573811020, 1573811030, 1573811040, 1573811050, 1573811060, 1573811070, 1573811080, 1573811090, 1573811100, 1573811110, 1573811120, 1573811130, 1573811140, 1573811150, 1573811160, 1573811170, 1573811180, 1573811190, 1573811200, 1573811210, 1573811220, 1573811230, 1573811240, 1573811250, 1573811260, 1573811270, 1573811280, 1573811290, 1573811300, 1573811310, 1573811320, 1573811330, 1573811340, 1573811350, 1573811360, 1573811370, 1573811380, 1573811390, 157381

In [183]:
def interpolate(time_serie_row, measures_row):
    ureal_list=[]
    for i in range(min(len(measures_row),len(time_serie_row))-1):
            a=0
            if (time_serie_row[i]-time_serie_row[i+1]) == 0:
                i=i+1
            else:
                b=(float(measures_row[i])-float(measures_row[i+1]))/(time_serie_row[i]-time_serie_row[i+1])
                c=float(measures_row[i])-time_serie_row[i]*(float(measures_row[i])-float(measures_row[i+1]))/(time_serie_row[i]-time_serie_row[i+1])
            r=False
            new_ureal = [a,b,c,r,time_serie_row[i],time_serie_row[i+1]]
            ureal_list.append(new_ureal)
    return Row(one_participant_profile = ureal_list)

# testing interpolate function 



list_time = list(PM100_test.select('time').head())[0]

list_measure = list(PM100_test.select('PM1_0').head())[0]
result = interpolate (list_time, list_measure)


def minimal_interval(row):
    diff_values=[]
    for i in range(len(row)-1):
        diff_values.append((row[i+1]-row[i]))
    return min(diff_values)

def traj_duration(row):
    return (max(row)-min(row)).seconds





In [171]:
# example for a row in the Dataframe
toto = spark.createDataFrame(sc.parallelize([result]),MrealType).head()
print(toto)

Row(time_units=[Row(a=0, b=-0.0, c=19.0, r=False, t1=1573810600, t2=1573810610), Row(a=0, b=-0.3, c=472143202.0, r=False, t1=1573810610, t2=1573810620), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810620, t2=1573810630), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810630, t2=1573810640), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810640, t2=1573810650), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810650, t2=1573810660), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810660, t2=1573810670), Row(a=0, b=-0.1, c=157381083.0, r=False, t1=1573810670, t2=1573810680), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810680, t2=1573810690), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810690, t2=1573810700), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810700, t2=1573810710), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810710, t2=1573810720), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810720, t2=1573810730), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810730, t2=1573810740), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810740, t2=15738107

In [172]:
type(str(type(list_time))),type(result),MrealType

(str,
 pyspark.sql.types.Row,
 StructType(List(StructField(time_units,ArrayType(StructType(List(StructField(a,IntegerType,true),StructField(b,DoubleType,true),StructField(c,DoubleType,true),StructField(r,BooleanType,true),StructField(t1,LongType,true),StructField(t2,LongType,true))),true),true))))

In [175]:
# profile temporelle d'un utilisateur 
PM100_test.select("participant_id").head(),result

(Row(participant_id=9999920),
 Row(one_participant_profile=[[0, -0.0, 19.0, False, 1573810600, 1573810610], [0, -0.3, 472143202.0, False, 1573810610, 1573810620], [0, -0.0, 16.0, False, 1573810620, 1573810630], [0, -0.0, 16.0, False, 1573810630, 1573810640], [0, -0.0, 16.0, False, 1573810640, 1573810650], [0, -0.0, 16.0, False, 1573810650, 1573810660], [0, -0.0, 16.0, False, 1573810660, 1573810670], [0, -0.1, 157381083.0, False, 1573810670, 1573810680], [0, -0.0, 15.0, False, 1573810680, 1573810690], [0, -0.0, 15.0, False, 1573810690, 1573810700], [0, -0.0, 15.0, False, 1573810700, 1573810710], [0, -0.0, 15.0, False, 1573810710, 1573810720], [0, -0.0, 15.0, False, 1573810720, 1573810730], [0, -0.0, 15.0, False, 1573810730, 1573810740], [0, -0.0, 15.0, False, 1573810740, 1573810750], [0, -0.0, 15.0, False, 1573810750, 1573810760], [0, -0.0, 15.0, False, 1573810760, 1573810770], [0, -0.0, 15.0, False, 1573810770, 1573810780], [0, -0.0, 15.0, False, 1573810780, 1573810790], [0, -0.1, 1573

In [184]:
udfInterpolate = udf(interpolate, MrealType)

udfMinimalInterval = udf(minimal_interval, IntegerType())

udfTrajDuration = udf(traj_duration, IntegerType())

In [185]:
PM100_test_profiles = PM100_test.withColumn('Ureal_data', udfInterpolate('time','PM1_0'))

In [186]:
PM100_test_minimal = PM100_test.withColumn('minimal', udfMinimalInterval('time'))

In [187]:
PM100_test_duration = PM100_test.withColumn('duration', udfTrajDuration('time'))

In [193]:
toto = PM100_test_profiles.select('participant_id','Ureal_data')

In [188]:
PM100_test_minimal.select('participant_id','minimal').show()

+--------------+-------+
|participant_id|minimal|
+--------------+-------+
|       9999920|     10|
|       9999955|     10|
|       9999975|      0|
|       9999960|     30|
|       9999964|     10|
|       9999962|     10|
+--------------+-------+



In [417]:
PM100_test4.select('participant_id','duration').show()

+--------------+--------+
|participant_id|duration|
+--------------+--------+
|       9999920|   43130|
|       9999955|   43130|
|       9999975|   42390|
|       9999960|   41530|
|       9999964|   42790|
+--------------+--------+



In [245]:
toto.printSchema()
a = toto.filter(toto['participant_id'] == 9999920).select('Ureal_data')
a.show()


root
 |-- participant_id: integer (nullable = true)
 |-- Ureal_data: struct (nullable = true)
 |    |-- time_units: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: integer (nullable = true)
 |    |    |    |-- b: double (nullable = true)
 |    |    |    |-- c: double (nullable = true)
 |    |    |    |-- r: boolean (nullable = true)
 |    |    |    |-- t1: long (nullable = true)
 |    |    |    |-- t2: long (nullable = true)

+--------------------+
|          Ureal_data|
+--------------------+
|[[[0, -0.0, 19.0,...|
+--------------------+



In [249]:
from pyspark.sql.functions import explode
# b = a.withColumn("array_elem", explode(a["Ureal_data"]["time_units"]))
# b.show()
# a.printSchema()
b = a.select(a['Ureal_data']['time_units'])
b.printSchema()
b.show()

root
 |-- Ureal_data.time_units: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- a: integer (nullable = true)
 |    |    |-- b: double (nullable = true)
 |    |    |-- c: double (nullable = true)
 |    |    |-- r: boolean (nullable = true)
 |    |    |-- t1: long (nullable = true)
 |    |    |-- t2: long (nullable = true)

+---------------------+
|Ureal_data.time_units|
+---------------------+
| [[0, -0.0, 19.0, ...|
+---------------------+



In [250]:
b.collect()

[Row(Ureal_data.time_units=[Row(a=0, b=-0.0, c=19.0, r=False, t1=1573810600, t2=1573810610), Row(a=0, b=-0.3, c=472143202.0, r=False, t1=1573810610, t2=1573810620), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810620, t2=1573810630), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810630, t2=1573810640), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810640, t2=1573810650), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810650, t2=1573810660), Row(a=0, b=-0.0, c=16.0, r=False, t1=1573810660, t2=1573810670), Row(a=0, b=-0.1, c=157381083.0, r=False, t1=1573810670, t2=1573810680), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810680, t2=1573810690), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810690, t2=1573810700), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810700, t2=1573810710), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810710, t2=1573810720), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810720, t2=1573810730), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810730, t2=1573810740), Row(a=0, b=-0.0, c=15.0, r=False, t1=1573810740,

In [259]:
# toto = spark.createDataFrame(b,UrealType)
c = b.collect()[0]
c[0].show()
# spark.createDataFrame(c, UrealType).show()

AttributeError: 'list' object has no attribute 'show'

In [266]:
# print(c[0])
maListe = c[0]
